<a href="https://colab.research.google.com/github/virajvaidya/Australia-Energy-Demand-Forecasting/blob/main/QLDPIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.3.5

In [ ]:
!pip install plotly==5.4.0
!pip install chart_studio

In [ ]:
!pip install statsmodels==0.13.1

In [ ]:
!pip install pmdarima

In [5]:
import pandas       as pd
import numpy        as np
import pmdarima     as pm
from pmdarima       import pipeline
from pmdarima       import model_selection
from pmdarima       import preprocessing as ppc
from pmdarima       import arima
from matplotlib     import pyplot as plt
import seaborn      as sns

In [6]:
import plotly.graph_objs    as go
from plotly.offline                       import iplot
from plotly.subplots                      import make_subplots

In [7]:
sns.set_style('whitegrid')
pd.plotting.register_matplotlib_converters()
# Default figure size
sns.mpl.rc('figure',figsize=(16, 6))

In [8]:
from google.colab import files
import io
uploaded = files.upload()

Saving QLD.csv to QLD.csv


In [10]:
df = pd.read_csv('QLD.csv')

In [11]:
df= df[['DATE','TOTALDEMANDQLD','RRPQLD']]

In [12]:
df['DATE'] = df['DATE'].apply(lambda x: pd.Timestamp(x).strftime('%d/%m/%Y %H%M'))

In [13]:
df['DATE'] = df['DATE'].astype('datetime64[ns]')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70942 entries, 0 to 70941
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            70942 non-null  datetime64[ns]
 1   TOTALDEMANDQLD  70942 non-null  float64       
 2   RRPQLD          70942 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.6+ MB


In [15]:
data = df['TOTALDEMANDQLD']
train, test = model_selection.train_test_split(data, train_size=40000)

In [16]:
pipe = pipeline.Pipeline([
    ("fourier", ppc.FourierFeaturizer(m=12, k=4)),
    ("arima", arima.AutoARIMA(stepwise=True, trace=1, error_action="ignore",
                              information_criterion='bic',
                              seasonal=False,  # because we use Fourier
                              suppress_warnings=True))
])

In [17]:
pipe.fit(train)
print("Model fit:")
print(pipe)

Performing stepwise search to minimize bic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : BIC=inf, Time=106.57 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : BIC=515428.487, Time=3.44 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : BIC=465845.252, Time=8.86 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : BIC=489649.669, Time=36.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : BIC=515417.892, Time=36.10 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : BIC=465854.801, Time=8.49 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : BIC=465855.102, Time=10.53 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : BIC=465600.489, Time=63.75 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : BIC=inf, Time=122.79 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : BIC=464959.583, Time=21.41 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : BIC=476081.800, Time=46.22 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : BIC=464925.482, Time=56.71 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : BIC=470975.103, Time=48.67 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : BIC=464963.389, Time=102.19 sec
 ARIM

In [18]:
preds, conf_int = pipe.predict(n_periods=100, return_conf_int=True)
print("\nForecasts:")
print(preds)


Forecasts:
[5316.35916496 5148.35618865 5035.41545841 4972.26214924 4944.05042288
 4931.05315225 4921.66523352 4927.26900142 4943.51981135 4946.38075463
 4929.47522649 4898.62920786 4858.56375963 4822.93979608 4804.89481447
 4808.16055474 4828.70002473 4849.97097283 4864.67089496 4887.20650849
 4915.35905781 4926.58597314 4915.56105908 4888.84864543 4851.68879514
 4818.10724161 4801.49791612 4805.77281099 4827.02163516 4848.79119801
 4863.84160831 4886.62359156 4914.94931435 4926.29795029 4915.35859555
 4888.7063231  4851.58874885 4818.03691719 4801.44848772 4805.73807308
 4826.99722203 4848.77403742 4863.82954723 4886.61511938 4914.94335997
 4926.29375914 4915.35564319 4888.70424129 4851.58728052 4818.03588526
 4801.44776643 4805.7375724  4826.99687496 4848.77379337 4863.82937713
 4886.61500561 4914.94328091 4926.29369787 4915.3555938  4888.70420005
 4851.58724655 4818.03586157 4801.44775386 4805.73756989 4826.99687806
 4848.77379545 4863.82938005 4886.61501347 4914.94328732 4926.293